In [ ]:
import sys
sys.path.append("../build")

In [ ]:
import lib.parquet_table as pqt

In [ ]:
pt = pqt.ParquetTable("../data/trade/bitmex/XBTUSD/20221109.parquet")

In [ ]:
pt.print_stats()

In [ ]:
pt.column(0)

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame({ 
    "time": pt.column(0),
    "price": pt.column(3),
    "size": pt.column(2)
})

In [ ]:
def parse_time(x):
    x = datetime.datetime.fromtimestamp(x/1e6)
    return x.strftime('%H:%M')
    
df['time_bin'] = df['time'].apply(parse_time)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
tf = df.groupby('time_bin').agg({
    'size': np.sum,
    'price': lambda x: x.iloc[-1]
}).reset_index()
tf.head()

In [ ]:
fig,ax1 = plt.subplots()

In [ ]:
ax2 = ax1.twinx()

In [ ]:
tf.plot(x='time_bin', y=['size'], figsize=(16, 8))

In [ ]:
# fig,ax1 = plt.subplots()
# ax2 = ax1.twinx()
# ax2.set_autoscale_on(b=True)
tf.plot(x='time_bin', y=['price', 'size'], figsize=(16, 8), secondary_y=True)

In [ ]:
plt.hist(np.log(s),bins=100)

In [ ]:
plt.style.use('_mpl-gallery')

plt.plot(pt1.column(0), pt1.column(3), linewidth=.2, c='red', label='Price')
plt.plot(pt1.column(0), pt1.column(2), linewidth=.2, c='blue', label='Volume')
plt.xlabel("Time")

plt.legend()

plt.show()



In [ ]:
!{sys.executable} -m pip install pyarrow

In [ ]:
!{sys.executable} -m pip install fastparquet

In [ ]:
from fastparquet import ParquetFile
pf = ParquetFile("../data/trade/bitmex/XBTUSD/20221109.parquet")

In [ ]:
df = pf.to_pandas()